# Blocksec Challenge - Elias Leers

This exercise asks you to investigate a quintessential blocksec incident: a 51% attack.

You may know a lot about what this attack is, you may know very little.
Regardless, you'll need to figure it out.
Blocksec is on the cutting edge of cryptocurrency security -- no one on the team is expected to know everything,
and everyone on the team is expected to be able to learn continuously and investigate new issues,
building out their own new knowledge as well as clearly communicating what they've learned to their teammates.

If you need to learn more about 51% attacks, try these resources:

* https://en.bitcoin.it/wiki/Majority_attack
* https://blog.coinbase.com/ethereum-classic-etc-is-currently-being-51-attacked-33be13ce32de
* https://www.youtube.com/watch?v=LlXst4zECcU&feature=youtu.be

The file "reorg_data.json" contains an array of blocks.
Each block is taken from the EthereumClassic (ETC) blockchain during or around a 51% attack.
Blocks contain metadata along with a(possibly empty) array of transactions that are contained in the block.
Transactions are structured as JSON objects where the human readable fields provide sufficient information to
understand the content of the transaction for the purpose of this challenge.

Your task is to write scripts in your language of choice to parse this data and answer the questions below.
Please provide the code that you used to do this.

If you see something in the data that you don’t understand, try searching online.
A key tool in the blocksec arsenal is something called a block explorer -- one you could try for EthereumClassic is
[blockscout.com](https://blockscout.com).

## Process the data
### Data structuring

I started by creating some classes to encapsulate the block data from the provided JSON file.

In [1]:
import json
import datetime
import math

class Block:
    def __init__(self, blockdict):
        self.ticker = blockdict['ticker']
        self.hash = blockdict['block_hash']
        self.parent_hash = blockdict['parent_hash']
        self.block_height = int(blockdict['block_height'])
        self.time = int(blockdict['time'])
        self.transaction_type = blockdict['transaction_type']
        self.transactions = blockdict['transaction_objects']

    def get_iso_time(self):
        return datetime.datetime.fromtimestamp(self.time).isoformat()

    def __str__(self):
        return str(self.block_height)

    def __repr__(self):
        return str(self.block_height)

class Chain:
    def __init__(self):
        self.baseNode = None
        self.chain = []

    def add_block(self, block):
        if self.baseNode is None:
            self.baseNode = block
        elif block.parent_hash != self.chain[-1].hash and block.block_height != self.chain[-1].block_height + 1 :
            raise ValueError
        self.chain.append(block)

    def replace_last_block(self, block):
        del self.chain[-1]
        self.add_block(block)

    def has_faults(self):
        hash_error, height_error = [False, False]
        previous_hash, previous_height = [None, None]

        for ublock in self.chain:
            if previous_height is not None and ublock.block_height != previous_height + 1:
                height_error = True
                print(ublock.block_height, previous_height)
            if previous_hash is not None and ublock.parent_hash != previous_hash:
                hash_error = True
                print(ublock.block_height)
            previous_hash = ublock.hash
            previous_height = ublock.block_height
        return hash_error, height_error

    def get_latest_block(self):
        return self.chain[-1]

    def get_latest_block_parent(self):
        return self.chain[-2]

    def get_chain(self):
        return self.chain

class Transaction:
    def __init__(self, tx, block):
        self.tx_to = tx['details']['to']
        self.tx_from = tx['details']['from']
        self.value = int(tx['details']['value'], 16) * math.pow(10, -18)
        self.nonce = int(tx['details']['nonce'], 16)
        self.txid = tx['txid']
        self.block = block

### Parse the JSON

Next, I parsed the blocks and stored them in 2 chains:

* canon stores the version of the chain that includes the reorganization.
* orphan stores the fork that got replaced by the attackers.

I noticed that the dataset held duplicate blocks, so I created an offset to work with. I kept the rest just in case, but
never ended up using them.

In [2]:
with open('reorg_data.json') as f:
    data = json.load(f)

blocks = []
for block in data:
    blocks.append(Block(block))

blocks.sort(key=lambda x: x.block_height)

canon = Chain()
orphan = Chain()
orphan_started = None
attack_start = None
attack_end = None
for block in blocks:
    try:
        canon.add_block(block)
    except ValueError:
        if orphan_started is None:
            orphan_started = canon.get_latest_block_parent()
            attack_start = len(canon.chain) -1
        try:
            orphan.add_block(canon.get_latest_block())
            canon.replace_last_block(block)
        except ValueError:
            print('This should not happen.')
        attack_end = len(canon.chain) -1

### Data integrity check

I added a function to the chain class to ensure that the proper blocks were inserted into the Canon chain.
This checks that each block's `parent_hash` and `block_height` matches the previous block's `hash` and `block_height + 1`

In [3]:
print('Canon has data faults: %s' % str(canon.has_faults()))
print('Orphan has data faults: %s' % str(orphan.has_faults()))

Canon has data faults: (False, False)
Orphan has data faults: (False, False)


I then find the point on the canon chain where it forks from the orphan chain with an additional offset notated as
`attack_actual_start_offset`.

In [4]:
attack_actual_start_offset = 0
for i in range(0, len(orphan.chain)):
    a = canon.chain[attack_start + i]
    b = orphan.chain[i]

    if a.hash != b.hash:
        attack_actual_start_offset = i
        break

## Reorg Analysis

### Briefly describe what happened.

On 2020-08-05 at 04:50:20, an attacker secretly mined the first block of their attack chain (block 10935623).

~15 hours later, after secretly mining ~4345-4352 blocks, they shared their work with the rest of the network.

This orphaned the network's 4243 blocks (blocks 10935623 to 10939866), because the attack chain was ~102-109 blocks
ahead of what the network had.

### What is the highest block height that was orphaned? What is its block hash?

This is the last block in the orphan chain.

In [5]:
last_orphan = orphan.chain[-1]
print('The last orphaned block is: %s' % last_orphan)
print("The last orphaned block's hash is: %s" % last_orphan.hash)

The last orphaned block is: 10939866
The last orphaned block's hash is: 0x955c4f106d1008b7fe0e8f32b3e811e3fc8272e4e03b05c326edb13097fa6678


### What block height was the last block that wasn't impacted by the reorg? (a.k.a. the "common ancestor" of the two chains.) What is its block hash?

I determined this by finding the two Blocks that share a `block_height` and `parent_hash`, but have different `hash`.

In [6]:
canon_attack_offset = attack_start + attack_actual_start_offset
ancestor = canon.chain[canon_attack_offset - 1]
orphan_first_block = orphan.chain[attack_actual_start_offset]
canon_first_block = canon.chain[canon_attack_offset]

orphan_first_block_ancestor_check = orphan_first_block.parent_hash == ancestor.hash
canon_first_block_ancestor_check = canon_first_block.parent_hash == ancestor.hash

chain_forks =  orphan_first_block.hash != canon_first_block.hash

ancestor_valid = orphan_first_block_ancestor_check and canon_first_block_ancestor_check and chain_forks
if ancestor_valid:
    print("The common ancestor's block height is %s" % ancestor)
    print("The common ancestor's hash is %s" % ancestor.hash)
else:
    print("This paper has a terrible error.")

The common ancestor's block height is 10935622
The common ancestor's hash is 0x092d1ea9e4e6a431f23e18feb72a4009df7df72955b24a14e4fc7d80bcd29cce


### How many blocks were orphaned?

This is the difference between the heights of last orphaned block, and the first block of the attack chain.

In [7]:
orphan_last_block = orphan.chain[-1]
total_orphaned = orphan_last_block.block_height - orphan_first_block.block_height
print('%s blocks were orphaned.' % total_orphaned)

4243 blocks were orphaned.


### How many blocks did the attacker create?  How do you know?

The first block the attackers created is the first block that was affected by the reorganization,
i.e. the first attack chain block (`attacker_first_block`).

Based on the dataset, I can't be sure if it includes more blocks that were mined after the reorg happened;
but I can infer that the last guaranteed block the attackers created is the first block created with a date higher than the last orphaned block
(`orphan_last_block`).

Any blocks afterwards could have been mined by attackers, or by the network after the reorg, so this number could be slightly higher.

In [8]:
attacker_first_block = canon.chain[canon_attack_offset]

attacker_last_block = None
for b in canon.chain:
    if b.time > orphan_last_block.time:
        attacker_last_block = b
        break

attacker_last_potential_block = canon.chain[-1]

# add 1 for inclusivity
total_blocks_created = attacker_last_block.block_height - attacker_first_block.block_height + 1
total_potential_blocks_created = attacker_last_potential_block.block_height - attacker_first_block.block_height + 1

extra_blocks_created = attacker_last_block.block_height - orphan_last_block.block_height + 1
extra_potential_blocks_created = attacker_last_potential_block.block_height - orphan_last_block.block_height + 1

print("First block of attack chain: %s" % attacker_first_block)
print("Last block of attack chain:  %s" % attacker_last_block)
print('Total blocks guaranteed created by attacker:  %s' % total_blocks_created)
print('Total blocks potentially created by attacker: %s' % total_potential_blocks_created)
print('Blocks above orphaned chain: %s to %s ' % (extra_blocks_created, extra_potential_blocks_created))

First block of attack chain: 10935623
Last block of attack chain:  10939968
Total blocks guaranteed created by attacker:  4346
Total blocks potentially created by attacker: 4353
Blocks above orphaned chain: 102 to 109 


### Can you estimate how long the attacker was mining the attack chain?  How do you know?

We have two potential methods:

* Ethereum Classic's block time is ~15 seconds.
If I multiply the number of attacker created blocks by 15 seconds, I can get a rough estimate.

* The other option is a little more accurate.
It appears that the attacker didn't spoof the block mining times,
so I can just calculate the offset between the start and end times of the attack.

In [9]:
start = canon.chain[canon_attack_offset]
end = orphan.chain[-1]

block_number_delta = datetime.timedelta(seconds=total_blocks_created * 15)
block_time_delta = datetime.timedelta(seconds=end.time - start.time)

print('The attack started at: %s' % start.get_iso_time())
print('The attack ended at:  %s' % end.get_iso_time())

print("The attack's estimated duration time was:")

print(" - Block number calculation: %s" % block_number_delta)
print(" - Block time calculation:   %s" % block_time_delta)

The attack started at: 2020-08-05T04:50:20
The attack ended at:  2020-08-05T20:14:15
The attack's estimated duration time was:
 - Block number calculation: 18:06:30
 - Block time calculation:   15:23:55


## Double spend analysis

### Process the transactions

In [10]:
attack_chain = canon.chain[canon_attack_offset:]
canon_tx = []
attack_tx = []
orphan_tx = []

for block in canon.chain:
    for tx in block.transactions:
        canon_tx.append(Transaction(tx, block))
for block in attack_chain:
    for tx in block.transactions:
        attack_tx.append(Transaction(tx, block))
for block in orphan.chain[attack_actual_start_offset:]:
    for tx in block.transactions:
        orphan_tx.append(Transaction(tx, block))


### How do you determine whether one transaction is double spending another?

When a reorganization occurs, all orphaned transactions get sent back to the network.
An attacker can block an orphaned transaction from completing by making it invalid.

The following conditions can invalidate an otherwise valid transaction:

* The transaction comes from an address that has already sent a transaction with the same nonce or higher.
This feature normally blocks the same transaction from accidentally running twice.
* The transaction comes from an address that doesn't have a high enough balance to cover the transaction.
(By using a block explorer, I was able to confirm that the attacker's double spends emptied the accounts since the
dataset doesn't show address balances.)

Any transaction in the attack chain where the following conditions are satisfied by a transaction in the orphaned chain
indicates a double spend attack:

* the tx_from address matches
* the nonce matches
* the tx_to address does not match

Additionally, any instance of the above where the attack transaction's value is less than the orphaned transaction value
may be spent across multiple transactions. I need to check for more orphaned transactions that spend from that address.

In [11]:
double_spends = []
for txa in attack_tx:
    for txo in orphan_tx:
        if txo.tx_from == txa.tx_from and txo.nonce == txa.nonce and txo.tx_to != txa.tx_to:
            double_spends.append({'attack': txa, 'orphan': txo})
num_double_spends = len(double_spends)

potential_multi_transaction_attacks = []
for txx in double_spends:
    if txx['attack'].value != txx['orphan'].value:
        potential_multi_transaction_attacks.append(txx)

over_nonce_spends = []
for txm in potential_multi_transaction_attacks:
    for txo in orphan_tx:
        if txo.tx_from == txm['attack'].tx_from and txo.nonce > txm['attack'].nonce:
            over_nonce_spends.append(txo)

print('There are %s transactions that double spend orphaned transactions.' % num_double_spends)


There are 7 transactions that double spend orphaned transactions.


### What addresses are controlled by the attacker?

The attacker controls the tx_from and tx_to addresses of all double spending transactions on the attack chain.

In [12]:
double_spending_txs = []
for tx in double_spends:
    double_spending_txs.append(tx['attack'])

attacker_addresses = []
exfil_addresses = []
for tx in double_spending_txs:
    if tx.tx_from not in attacker_addresses:
        attacker_addresses.append(tx.tx_from)
    if tx.tx_to not in exfil_addresses:
        exfil_addresses.append(tx.tx_to)

print('The attacker controls the following addresses:')
print('Attacking addresses:')
for address in attacker_addresses:
    print(address)
print('Exfiltration addresses:')
for address in exfil_addresses:
    print(address)

The attacker controls the following addresses:
Attacking addresses:
0xa56cfaef495a45f17f44fd0b2d85e0fe63b9ba7d
0x4b77501d21c3c3488c60fcea640b590be321bc5f
0xc39241a847c550bbef4489653b1006b980709e4b
0x806a915693c2f83883f5be2c6bb2d7174033af4a
0x7946fc0611fe2e22f90ab3d196605890ae2096ac
0xe1827a7ce46bf5977fe5a38c94f1b2d1ce751981
0x1fb24ec5da95f0438d8b16e05285455c8079e1b3
Exfiltration addresses:
0x9bcf1d5facbb67a35fb84dd7eafd25f4486cb265
0x8bc7cc58c3f5449e1840f4267ca214b3a9b629f8
0xc1ae4099a747466ec1c0cddb6e772e1e6896cc6b
0x1b0176245209a38c3ac7a8263039256a39cd1263
0x3c821b3a28b0aa9690b864c2de0b4224a903d79b


### How many orphaned transactions were double spent? What are their Transaction IDs(TXIDs)? What is their total value? What blocks were they in?

In [13]:
double_spent_orphan_txs = []
for tx in double_spends:
    double_spent_orphan_txs.append(tx['orphan'])

for tx in over_nonce_spends:
    double_spent_orphan_txs.append(tx)

total_value = 0
for tx in double_spent_orphan_txs:
    total_value += tx.value

print('%s orphaned transactions were double spent.\n' % len(double_spent_orphan_txs))
print("{:<66} {:<17} {:<8}".format('Transaction ID','Value','Block'))
for tx in double_spent_orphan_txs:
    print("{:<66} {:<17} {:<8}".format(tx.txid, tx.value, str(tx.block)))

print('\nThe attackers double spent %s ETC' % total_value)

9 orphaned transactions were double spent.

Transaction ID                                                     Value             Block   
0xdd6dc3537c72bcc26f7ca970eaedb1497df7e541f3c5338f54e5ed8466eec990 10.0              10935682
0x658ffdf30a859f437422ee0cd4b4182021cc67c9c412b115b10ad820157ac9a4 97709.99979000002 10936604
0x0452e4c2ae7fbc38f90ddabd767b4778c7623633b795385f72f7f9bda0900889 84488.0           10937084
0x8e099dfd6e1044785f7b0ac3dd2f0fccf3a2ae4f876dfda19f930b919b377e48 7908.000000000001 10937561
0xea89573b8c0d446f7594af2cb5b5d33966d5d301ea8190faaa7220bafd70955a 34056.0           10937576
0xfe177fff4e37a5d0e275ecab26f24f12c55875a6451914bd5f5f59e5906aca63 73402.0           10937572
0x148681d845854c2d2b4ffc2105d21d3a6bb1585e82e5b94d03776fe912719f5a 13794.0           10937587
0x1bf98fd21c22fb99a7ac797a20ce8ad939cd78c61b8d55cd157828945015080a 82120.0           10935694
0xaf56ca61d94fe859435e91e40e65ddb73383c60aada22681f65ad27f1b15f514 71959.99829270001 10936006

The attackers d

### Which transactions did the double spending of the orphaned transactions provided above? Is it the same number of transactions? Why or why not?

In [14]:
print('Transactions that double spend orphaned transactions:')
print("{:<66} {:<17} {:<8}".format('Transaction ID','Value','Block'))
for tx in double_spending_txs:
    print("{:<66} {:<17} {:<8}".format(tx.txid, tx.value, str(tx.block)))

double_spend_tx_amount_check = len(double_spent_orphan_txs) == len(double_spending_txs)
print('\n Number of double spending transactions matches spent transactions: %s' % double_spend_tx_amount_check)

Transactions that double spend orphaned transactions:
Transaction ID                                                     Value             Block   
0x2e287be99687b57cfe678e4d058448b5d40576e7240e8003bea1851061b49860 154088.0          10935680
0x7ab164f8ba72d8054ded132d8f6983f8914dd3f9f7a5592b48a521b397f19d88 97708.0           10936067
0x3ec37e8d3b52efb45e4e1c8a6962b72452891e83391566b11698d7b58f316ba1 84488.0           10936157
0xea152319f7614efb68d4aaec998607e28012aed84160d69d91eb41767a0dfc82 7908.000000000001 10936264
0x90a8d2013b1d2ae03a09b4ba70243ae034e1345949eaed3b3ad27f8e4d30b384 34056.0           10936284
0xa07ecf134ac0a663aa13612e91229e4b75f1719142a3a5048da30fa52a91379e 73402.0           10936284
0xe572ae9eb28b0257355236fd6872753eadcb0747557fe98d59cf1e8d002492be 13794.0           10936286

 Number of double spending transactions matches spent transactions: False


It is not the same number of transactions because the attackers double spent 3 orphaned transactions with a single
transaction from address 0xa56cfaef495a45f17f44fd0b2d85e0fe63b9ba7d.
(I had to confirm with a block explorer that the balance was empty for this one.)

### What number of confirmations would an exchange have needed in order to be completely safe from this attack? Why?

The number of confirmations that would be needed to be safe can be calculated with the first double spent transaction's
block height and the last orphaned block's height.

In [15]:
double_spent_by_height = sorted(double_spent_orphan_txs, key=lambda l: l.block.block_height)
num_confirmations_safe = orphan.chain[-1].block_height - double_spent_by_height[0].block.block_height
print('To be completely safe from this attack, an exchange would need %s confirmations.' % num_confirmations_safe)

To be completely safe from this attack, an exchange would need 4184 confirmations.


## Recommendations for Coinbase

### How does this affect Coinbase?

1. Coinbase could be a potential target of future attacks if their confirmation requirement is low enough on smaller
blockchains.

2. Any holdings Coinbase has in vulnerable coins is at risk if another exchange is the target of a future attack.

### What steps should Coinbase take?

1. Coinbase should raise the number of confirmations for all coins and tokens that run on blockchains with a low hashrate.
Ideally this number should be at least several weeks, due to the availability of rented hashpower.

2. Coinbase should share any suspicions of double spend activity with other exchanges to lower the potential value lost
due to a post-attack price drop.

### What information or events should Coinbase engineers attempt to observe going forward based on this analysis?
This attack only took 15 hours to steal 465448 ETC, which on that date was \\$3,351,225 at \\$7.20 per ETC.
Assuming the attacker rented hashpower from a 3rd party provider, i.e. NiceHash, Coinbase should flag any deposits that
are larger than the cost to rent the entire hashrate of that network for more than 24 hours.

Coinbase should also monitor larger blockchains experiencing a sudden drop in hashrate, as that could signal an attack
brewing for a smaller chain, especially those that are mined primarily on GPUs instead of ASICs.